In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests


dt_links = []
for yr in [str(y) for y in range(2010, 2022)]:
    lpyr = 0
    if int(yr) % 4 == 0: lpyr = 1
    for mnth in [m for m in range(1, 13)]:
        if mnth in [4, 6, 9, 11]:
            dt_links.append(str(mnth) + '-' + '30' + '-' + str(yr))
        if mnth in [1, 3, 5, 7, 8, 10, 12]:
            dt_links.append(str(mnth) + '-' + '31' + '-' + str(yr))
        if mnth == 2 and lpyr == 0:
            dt_links.append(str(mnth) + '-' + '28' + '-' + str(yr))
        elif mnth == 2 and lpyr == 1:
            dt_links.append(str(mnth) + '-' + '29' + '-' + str(yr))

podcast_info = {'podcast_date': [], 'title': [], 
                'summary': [], 'href': [], 
                'transcript_href': []}

URL = 'https://www.npr.org/series/423302056/hidden-brain/archive?date='
for dt in dt_links:
    page = requests.get(URL + dt)
    soup = BeautifulSoup(page.content, 'html.parser')
    for x in soup.find_all("article", {'class': 'item has-image'}): 
        x.decompose()
    for item in soup.find_all('h2', {'class': 'title'}):
        title = item.find('a').text
        href = item.find('a')['href']
        podcast_info['title'].append(title)
        podcast_info['href'].append(href)
    for p in soup.find_all('p', {'class': 'teaser'}):
        podcast_date, summary = p.text.split('•')
        podcast_info['podcast_date'].append(podcast_date)
        podcast_info['summary'].append(summary)
    for x in soup.find_all('ul', {'class': 'audio-module-more-tools'}):
        transcript_class = x.find('li', {'class': 'audio-tool audio-tool-transcript'})
        if not transcript_class:
            podcast_info['transcript_href'].append(None)
        else:
            transcript_class_href = transcript_class.find('a')['href']
            podcast_info['transcript_href'].append(transcript_class_href)

df = pd.DataFrame.from_dict(podcast_info)
df = df.drop_duplicates()

df.to_csv('hidden_brain_podcast_summaries.csv', index=False)

In [2]:
df.iloc[200]

podcast_date                                     September 22, 2015 
title              Trying To Change, Or Changing The Subject? How...
summary             The first episode of Hidden Brain explores sw...
href               https://www.npr.org/2015/09/22/434597124/tryin...
transcript_href            https://www.npr.org/transcripts/434597124
Name: 680, dtype: object

In [5]:
def get_transcript(href):
    k += 1
    page = requests.get(href)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    transcript = []
    for x in soup.find_all('p'):
        transcript.append(x.text)
    
    return '\n'.join(transcript)

In [3]:
df['transcript'] = df['transcript_href'].map(lambda href: get_transcript(href) if href is not None else None)

In [8]:
df.to_csv('podcast_transcripts.csv', index=False)